In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
from dateutil.relativedelta import relativedelta
from functools import partial
import pandas as pd

dask.config.set(**{'array.slicing.split_large_chunks': True})

### Set up basic info

In [12]:
ystart = 1970 # start year of hindcasts 
yend = 2020 # end year of hindcasts
nyears = yend - ystart + 1
initmon=11
initmonstr = str(initmon).zfill(2)
nmems = 20 # the number of hindcast members
memstr = [str(i).zfill(3) for i in np.arange(1,nmems+1,1)] # generating member strings
topdir = "/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/"
expname = "b.e21.BSMYLE-CW3E.f09_g17"
outpath="/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/Tzm/"

### Set up the dask cluster

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=30GB',
    project='P04010022',
    walltime='03:00:00',
    interface='mgt')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/work/islas/conda-envs/islaenv/lib/python3.11/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/work/islas/conda-envs/islaenv/lib/python3.11/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/work/islas/conda-envs/islaenv/lib/python3.11/site

In [8]:
cluster

PBSCluster(8238c9e9, 'tcp://10.18.206.68:37161', workers=24, threads=24, memory=670.56 GiB)

### Set up the file list

In [13]:
files = []
for iyear in np.arange(ystart,yend+1,1):
    filest = [ sorted(glob.glob(topdir+expname+"."+str(iyear)+"-"+initmonstr+"."+imem+"/atm/proc/tseries/day_1/*.THzm.*"))[0] for imem in memstr ]
    files.append(filest)

### Set up pre-processor

In [14]:
# pre-processor to ensure all initialization dates have the same time axis
def preprocessor(ds):
    # sort out the times so that each member has the same time axis
    timebndavg = np.array(ds.time_bnds,
                         dtype = 'datetime64[s]').view('i8').mean(axis=1).astype('datetime64[s]')
    ds['time'] = timebndavg
    ds = ds.where( ds.time.dt.hour == 12, drop=True)
    
    datestart=pd.to_datetime("1970-"+str(ds.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(ds.isel(time=0).time.dt.day.values).zfill(2)+
                       "-"+str(ds.isel(time=0).time.dt.hour.values), format="%Y-%m-%d-%H")
    time = [ datestart + relativedelta(days=int(i)) for i in np.arange(0,ds.time.size,1) ]
    ds['time'] = time
    ds = ds.isel(zlon=0)
    return ds

In [15]:
dat = xr.open_mfdataset(files, combine='nested', concat_dim=['init_year','M'],
                        parallel=True, data_vars=['THzm'], coords='minimal', compat='override', preprocess=partial(preprocessor),
                       chunks={'M':20, 'time':365})
dat['init_year'] = np.arange(ystart,yend+1,1)
dat = dat.THzm*(dat.ilev/1000.)**(2./7.)
dat = dat.rename("Tzm")
dat.to_netcdf(outpath+"Tzm_BSMYLE-CW3E_day_init"+initmonstr+".nc")

In [16]:
cluster.close()

/glade/work/islas/conda-envs/islaenv/lib/python3.11/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/work/islas/conda-envs/islaenv/lib/python3.11/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
